In [ ]:
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
import tests
import numpy as np
from heapq import heappush, heappop

%matplotlib notebook


 # Tests

In [ ]:
from typing import Callable

test_list = [tests.case1, tests.case2, tests.case3, tests.case4]
test_list = list(map(lambda t: list(map(Move.from_str, t.split(" "))), test_list))
def test(algorithm: Callable[[Cube], list[Move]], tests: list[list[Move]]) -> bool:
    for idx, test in enumerate(tests):
        cube: Cube = Cube(test)
        path: list[Move] = algorithm(cube)
        for move in path:
            cube = cube.move(move)
        if not is_solved(cube):
            print(f"Test {idx} failed")
            break
        else:
            print(f"Test {idx} passed")

 # A*

In [ ]:
def is_solved(cube: Cube) -> bool:
    for i in range(len(cube.state)):
        if cube.state[i] != cube.goal_state[i]:
            return False
    return True

In [ ]:
def get_neighbors(cube: Cube) -> list[tuple[Cube, Move]]:
    return [(cube.move(move), move) for move in Move]

In [ ]:
def heuristic(cube: Cube) -> int:
    return np.sum(cube.state != cube.goal_state)

In [ ]:
def get_path(cube_hash: str, discovered: dict[str, tuple[str, Move, int]]) -> list[Move]:
    path: list[Move] = []
    currentNode = discovered[cube_hash]
    while currentNode[0] is not None:
        path.append(currentNode[1])
        currentNode = discovered[currentNode[0]]
    path.reverse()
    return path

In [ ]:
def a_star(cube: Cube) -> list[Move]:
    # initialize with cube
    frontier: list[tuple(int, str, Cube)] = []
    heappush(frontier, (0 + heuristic(cube), cube.hash(), cube.clone()))
    discovered: dict[str, tuple[str, Move, int]] = {cube.hash(): (None, None, 0)}
    # search
    while frontier:
        currentCube: Cube = heappop(frontier)[2]
        if is_solved(currentCube):
            break
        for (neighbor, move) in get_neighbors(currentCube):
            score: int = discovered[currentCube.hash()][2] + 1
            if neighbor.hash() not in discovered or score < discovered[neighbor.hash()][2]:
                discovered[neighbor.hash()] = (currentCube.hash(), move, score)
                node: tuple[int, str, Cube] = (score + heuristic(neighbor), neighbor.hash(), neighbor.clone())
                heappush(frontier, node)
    # get path
    return get_path(currentCube.hash(), discovered)

 # Test A*

In [ ]:
test(a_star, test_list)

 # Bidirectional BFS

In [ ]:
def met_in_the_middle(cubes1: dict[str, tuple[Cube, Move, int]], cubes2: dict[str, tuple[Cube, Move, int]]) -> str:
    for key in cubes1:
        if key in cubes2:
            return key
    return None

In [ ]:
from collections import deque

def bidirectional_bfs(cube: Cube) -> list[Move]:
    frontier1: deque[Cube] = deque()
    frontier1.append(cube)
    discovered1: dict[str, tuple[str, Move, int]] = {cube.hash(): (None, None, 0)}

    solved_cube = cube.clone()
    solved_cube.state = solved_cube.goal_state
    frontier2: deque[Cube] = deque()
    frontier2.append(solved_cube)
    discovered2: dict[str, tuple[str, Move, int]] = {solved_cube.hash(): (None, None, 0)}

    while frontier1 and frontier2:
        met_cube_key: str = met_in_the_middle(discovered1, discovered2)
        if met_cube_key is not None:
            break
        currentCube1: Cube = frontier1.popleft()
        currentCube2: Cube = frontier2.popleft()
        for (neighbor, move) in get_neighbors(currentCube1):
            score: int = discovered1[currentCube1.hash()][2] + 1
            if neighbor.hash() not in discovered1 or score < discovered1[neighbor.hash()][2]:
                discovered1[neighbor.hash()] = (currentCube1.hash(), move, score)
                frontier1.append(neighbor)
        for (neighbor, move) in get_neighbors(currentCube2):
            score: int = discovered2[currentCube2.hash()][2] + 1
            if neighbor.hash() not in discovered2 or score < discovered2[neighbor.hash()][2]:
                discovered2[neighbor.hash()] = (currentCube2.hash(), move, score)
                frontier2.append(neighbor)
    path1: list[Move] = get_path(met_cube_key, discovered1)
    path2: list[Move] = get_path(met_cube_key, discovered2)
    path2.reverse()
    path2 = list(map(Move.opposite, path2))
    return path1 + path2

 # Test Bidirectional BFS

In [ ]:
test(bidirectional_bfs, test_list)